# 搭建并使用向量数据库

## 前序数据预处理

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# 批量处理文件夹中所有文件
import os

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '../data_base/knowledge_path/VMAX-S'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths)

from langchain.document_loaders.pdf import PyMuPDFLoader
# from langchain.document_loaders.markdown import UnstructuredMarkdownLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    else:
        print(f"Unsupported file type: {file_type} for file {file_path}")

# 下载文件并存储到text
# 加载所有文档内容到 texts
texts = []
for loader in loaders:
    texts.extend(loader.load())  # 关键步骤：初始化 texts

    
# 作数据清洗
# 修改后的数据清洗部分（替换原始代码中对应段落）
import re

# 预编译正则表达式（提升效率）
linebreak_pattern = re.compile(
    r'(?<![\\u4e00-\\u9fff])\n(?![\\u4e00-\\u9fff])',  # 负向断言匹配非中文环境换行
    flags=re.DOTALL
)
space_pattern = re.compile(r'[ 　]+')  # 匹配半角/全角空格
special_chars = ['•', '▪', '▫', '▶', '®', '©']  # 可扩展的干扰符号列表

# 替换原始代码中的清洗循环
for text in texts:
    # 1. 清理非中文环境换行
    text.page_content = re.sub(
        linebreak_pattern,
        lambda m: m.group().replace('\n', ''),
        text.page_content
    )

    # 2. 批量清理特殊符号
    for char in special_chars:
        text.page_content = text.page_content.replace(char, '')

    # 3. 安全删除空格（保留URL等特殊场景）
    text.page_content = space_pattern.sub('', text.page_content)



from dotenv import load_dotenv, find_dotenv 
# pip install python-dotenv

_ = load_dotenv(find_dotenv())

from langchain_community.embeddings import ZhipuAIEmbeddings

zhipu_embed = ZhipuAIEmbeddings(
    model="embedding-2",
    api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io",
)


''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter



# 知识库中单段文本长度
CHUNK_SIZE = 512

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50


# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)


split_docs = text_splitter.split_documents(texts)
print(f"切分后的文件数量：{len(split_docs)}")



print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")



split_docs[90].page_content



## 前序embedding模型

ZhipuAIEmbeddings模型

In [ ]:
# embedding选型
from langchain_community.embeddings import OllamaEmbeddings
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

批量处理文件夹中所有文件

## 构建Chroma向量库

涉及到的API

直接调用Chroma.from_documents (推荐)

本地

In [ ]:
# from langchain_milvus import Milvus
from langchain_community.vectorstores import Milvus

URI = "../milvus_example.db"

vector_store = Milvus(
    embedding_function=embeddings,
    connection_args={"uri": URI},
    index_params={"index_type": "FLAT", "metric_type": "L2"},
)

In [ ]:
persist_directory="../milvus_example.db"

vectordb = Chroma.from_documents(
        documents=split_docs,  # 为了速度，只选择前 20 个切分的 doc 进行生成
        embedding=emb_bgem3,
        collection_name="vmax-s", # 如果不指定默认为langchain
        persist_directory=persist_directory, # 允许我们将persist_directory目录保存到磁盘上
    )

服务端

In [ ]:
# from langchain_milvus import Milvus
from langchain_community.vectorstores import Milvus
from langchain_core.documents import Document  # 导入 Document 类


# 向量库创建
connection_args = {
    "host": "192.168.0.188",
    "port": "19530",
}

split_docs = [
    Document(page_content="你好", metadata={"source": "example"}),
    # 可以添加更多文档...
]


vectordb = Milvus.from_documents(
    documents=split_docs,
    embedding=emb_bgem3,
    collection_name="vmaxs",
    drop_old=False,
    connection_args=connection_args,
)

In [ ]:
split_docs[1]

In [ ]:
# 汇总

import os
from langchain_community.vectorstores import Chroma
# from langchain_chroma.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
# persist_directory = '../milvus-vmax'

# 向量库创建
connection_args = {
    "host": "192.168.0.188",
    "port": "19530",
}


# 创建嵌入模型
# embedding = ZhipuAIEmbeddings(model="embedding-2", api_key = os.environ['ZHIPUAI_API_KEY'])
# embedding选型
from langchain_community.embeddings import OllamaEmbeddings
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 初始化 Chroma 向量数据库
    # vectordb = Chroma.from_documents(
    #     documents=split_docs[0:3],  # 为了速度，只选择前 20 个切分的 doc 进行生成
    #     embedding=emb_bgem3,
    #     collection_name="vmax-s", # 如果不指定默认为langchain
    #     persist_directory=persist_directory, # 允许我们将persist_directory目录保存到磁盘上
    # )
    vectordb = Milvus.from_documents(
    documents=split_docs[0:3],
    embedding=emb_bgem3,
    collection_name="vmaxs",
    drop_old=False,
    connection_args=connection_args,
    )
    
    # 持久化向量数据库
    # vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

以上代码会报错： 持久化过程中发生错误: Error code: 400, with error text {"error":{"code":"1214","message":"input数组最大不得超过64条"}}

建议分批次导入

In [ ]:
import os
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings

# 定义持久化目录
# persist_directory = '../chroma-vmax'
# 向量库创建
connection_args = {
    "host": "192.168.0.188",
    "port": "19530",
}


# 创建嵌入模型
from langchain_community.embeddings import OllamaEmbeddings
emb_bgem3 = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

# 定义每批处理的文档数量
batch_size = 30

try:
    # 计算总批次数
    total_batches = (len(split_docs) + batch_size - 1) // batch_size
    
    # 初始化向量数据库（如果是第一次创建）
    vectordb = None
    
    for batch_num in range(total_batches):
        # 计算当前批次的起始和结束索引
        start_idx = batch_num * batch_size
        end_idx = min((batch_num + 1) * batch_size, len(split_docs))
        
        # 获取当前批次的文档
        batch_docs = split_docs[start_idx:end_idx]
        
        print(f"正在处理第 {batch_num + 1}/{total_batches} 批文档 (文档 {start_idx}-{end_idx-1})")
        
        if batch_num == 0:
            # 第一次创建向量数据库
            vectordb = Milvus.from_documents(
            documents=split_docs[0:3],
            embedding=emb_bgem3,
            collection_name="vmaxs",
            drop_old=False,
            connection_args=connection_args,
            )
    
        else:
            # 后续批次添加到现有集合
            vectordb.add_documents(batch_docs)     
        # 每批处理后持久化
        # vectordb.persist()
        print(f"第 {batch_num + 1} 批文档已成功导入并持久化")
    
    print("所有文档已成功导入并持久化到向量数据库。")
    
except Exception as e:
    print(f"处理过程中发生错误: {e}")

charma服务器版本

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.environ['ZHIPUAI_API_KEY'],
                model_name="embedding-2"
            )
collection = chroma_client.get_or_create_collection(
    name="test4",
    embedding_function=openai_ef,  # 使用openai
    metadata={
        "hnsw:space": "cosine", 
        "hnsw:construction_ef": 150,
        "hnsw:M": 24,
        "hnsw:num_threads": 8
    }
)
ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
print(ids_list)

try:
    # 添加文档到集合
    collection.add(documents=split_docs[:20], ids =ids_list )
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

通过chroma数据库add

按官网文档进行操作

In [ ]:
content_list = ['本发布日期南书', 
                '前言周志华老师的《机器学习》西瓜书是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能不太友好，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周老师之所以省去这些推导细节的真实原因是，他本尊认为理工科数学基础扎实点的大二下学生应该对西瓜书中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习。所以本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的理工科数学基础扎实点的大二下学生。使用说明南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第章和第章的公式强烈不建议深究，简单过一下即可，等你学得', 
                '有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力争以本科数学基础的视角进行讲解，所以超纲的数学知识我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们的地址：进行反馈，在对应版块提交你希望补充的公式编号或者勘误信息，我们通常会在小时以内给您回复，超过小时未回复的话可以微信联系我们微信号：；配套视频教程：在线阅读地址：仅供第版最新版获取地址：编委会', 
                '编委会主编：、、编委：、、、、封面设计：构思、创作林王茂盛致谢特别感谢、、、、、、、、、、、在最早期的时候对南瓜书所做的贡献。扫描下方二维码，然后回复关键词南瓜书，即可加入南瓜书读者交流群版权声明本作品采用知识共享署名非商业性使用相同方式共享国际许可协议进行许可。'
               ]
print(len(content_list))

In [ ]:
collection.add(
    documents=["doc1", "doc2", "doc3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    ids=["id1", "id2", "id3", ...]
)


In [ ]:
from chromadb.utils import embedding_functions
import os
# from langchain_community.vectorstores import Chroma
import chromadb
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = '../chroma-vmax'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.environ['ZHIPUAI_API_KEY'],
                model_name="embedding-2"
            )
# collection = client.get_or_create_collection(
#     name="test",
#     embedding_function=openai_ef,  # 使用openai
#     metadata={
#         "hnsw:space": "cosine", 
#         "hnsw:construction_ef": 150,
#         "hnsw:M": 24,
#         "hnsw:num_threads": 8
#     }
# )

# HNSW索引构建参数优化
collection = client.create_collection(
    name="high_perf",
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)

content_list = ['本发布日期南书', 
                '前言周志华老师的《机器学习》西瓜书是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读者通过西瓜书对机器学习有所了解所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推导细节的读者来说可能不太友好，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充具体的推导细节。读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周老师之所以省去这些推导细节的真实原因是，他本尊认为理工科数学基础扎实点的大二下学生应该对西瓜书中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习。所以本南瓜书只能算是我等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的理工科数学基础扎实点的大二下学生。使用说明南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第章和第章的公式强烈不建议深究，简单过一下即可，等你学得', 
                '有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力争以本科数学基础的视角进行讲解，所以超纲的数学知识我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，或者你发现南瓜书哪个地方有错误，请毫不犹豫地去我们的地址：进行反馈，在对应版块提交你希望补充的公式编号或者勘误信息，我们通常会在小时以内给您回复，超过小时未回复的话可以微信联系我们微信号：；配套视频教程：在线阅读地址：仅供第版最新版获取地址：编委会', 
                '编委会主编：、、编委：、、、、封面设计：构思、创作林王茂盛致谢特别感谢、、、、、、、、、、、在最早期的时候对南瓜书所做的贡献。扫描下方二维码，然后回复关键词南瓜书，即可加入南瓜书读者交流群版权声明本作品采用知识共享署名非商业性使用相同方式共享国际许可协议进行许可。'
               ]
# print(len(content_list))

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
metadatas = [{"source": "权威文档.pdf"} for _ in range(len(split_docs[:20]))]

ids_list = [f"doc_{i+1}" for i in range(len(content_list))]
# print(ids_list)


# print(metadatas)

try:
    # 添加文档到集合（修正后的参数）
    collection.add(
        documents=content_list,  # 传入纯文本列表
        metadatas=[{"source": "权威文档.pdf"},{"source": "权威文档.pdf"},{"source": "权威文档.pdf"},{"source": "权威文档.pdf"} ],
        ids=ids_list
    )
    print("数据写入成功，已自动持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")
except chromadb.errors.ChromaError as ce:
    print(f"数据库错误: {ce}")
except ValueError as ve:
    print(f"数据格式错误: {ve}")

正式方法

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    base_url='http://localhost:11434',
    model="nomic-embed-text:latest"
)
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r2 = ollama_emb.embed_query(
    "What is the second letter of Greek alphabet"
)

# 打印结果
print("Document embeddings:")
for i, embedding in enumerate(r1):
    print(f"Document {i+1} embedding: {embedding[:10]}")

print("\nQuery embedding:")
print(f"Query embedding: {r2[:10]}")

In [ ]:
from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
import requests

ollama_ef = OllamaEmbeddingFunction(
    url="http://localhost:11434",
    model_name="qwen2.5:0.5b "
)

try:
    response = ollama_ef._session.post(
        ollama_ef._api_url,
        json={"model": "nnomic-embed-text:latest", "prompt": "Test"}
    )
    print("原始响应内容:", response.text)  # 打印原始响应
    embeddings = ollama_ef(["Test"])
except Exception as e:
    print(f"错误详情: {str(e)}")

In [ ]:
from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434",model_name="nomic-embed-text:latest")
embeddings = ollama_ef("This is my first text to embed")

In [ ]:
from chromadb.utils import embedding_functions
import os
# from langchain_community.vectorstores import Chroma
import chromadb
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
# 定义持久化目录
persist_directory = '../chroma-vmax-3'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

from chromadb.utils.embedding_functions.ollama_embedding_function import OllamaEmbeddingFunction
ollama_ef = OllamaEmbeddingFunction(url="http://localhost:11434",model_name="nomic-embed-text:latest")

collection = client.get_or_create_collection(
    name="vmax-s",
    embedding_function=ollama_ef,  # 可选，但建议保持一致性
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)

# content_list = [doc.page_content for doc in split_docs[:20]]
content_list = [doc.page_content for doc in split_docs]

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
# metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs[:20]))]
metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs))]

# ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
ids_list = [f"doc_{i+1}" for i in range(len(split_docs))]
# print(ids_list)


# print(metadatas)

try:
    # 添加文档到集合（修正后的参数）
    collection.add(
        documents=content_list,  # 传入纯文本列表
        metadatas=metadatas, 
        ids=ids_list
    )
    print("数据写入成功，已自动持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")
except chromadb.errors.ChromaError as ce:
    print(f"数据库错误: {ce}")
except ValueError as ve:
    print(f"数据格式错误: {ve}")

通过chroma数据库add-改良版 
每次三个doc，分批次导入

In [ ]:
# 汇总

import os
# from langchain.vectordbs.chroma import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import ZhipuAIEmbeddings
from chromadb import PersistentClient
from chromadb import Documents, EmbeddingFunction, Embeddings
# 定义持久化目录
persist_directory = '../chroma-vmax-batch'
# 初始化客户端（持久化模式）
client = PersistentClient(path=persist_directory)

# 创建嵌入模型
# import chromadb.utils.embedding_functions as embedding_functions
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#                 api_key = os.environ['ZHIPUAI_API_KEY'],
#                 model_name="embedding-2"
#             )

# import chromadb.utils.embedding_functions as embedding_functions
# cohere_ef  = embedding_functions.CohereEmbeddingFunction(api_key="Tahx1eySFbKvu9sTyTXrRLf59la3ZUG9vy02stRZ",  model_name="embed-multilingual-v3.0")

zhipu_ef = ZhipuAIAdapter(model_name="embedding-2", api_key="5713143e8fdc4b4a8b284cf97092e70f.qEK71mGIlavzO1Io")

# 创建集合并配置HNSW参数
# collection = client.create_collection( 
# collection = client.get_or_create_collection(
#     name="test4",
#     embedding_function=openai_ef,
#     metadata={
#         "hnsw:space": "cosine", 
#         "hnsw:construction_ef": 150,
#         "hnsw:M": 24,
#         "hnsw:num_threads": 8
#     }
# )

collection = client.get_or_create_collection(
    name="vmax-s",
    embedding_function=cohere_ef,  # 可选，但建议保持一致性
    metadata={
        "hnsw:construction_ef": 200,  # 构建阶段搜索范围（默认100）
        "hnsw:search_ef": 320,        # 查询阶段搜索范围 
        "hnsw:M": 48                  # 图节点最大连接数
    }
)
# content_list = [doc.page_content for doc in split_docs[:20]]
content_list = [doc.page_content for doc in split_docs]

# 如果每个文档块的元数据需要个性化（例如记录分块编号），可以生成与文档数量匹配的元数据列表：
# metadatas = [{"source": "权威文档.pdf", "chunk_id": i} for i in range(len(split_docs[:20]))]

# 如果所有文档块的元数据相同（例如来源一致），可通过列表推导式生成等长元数据列表：
# metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs[:20]))]
metadatas = [{"source": "VMAX安装系列.pdf"} for _ in range(len(split_docs))]

# ids_list = [f"doc_{i+1}" for i in range(len(split_docs[:20]))]
ids_list = [f"doc_{i+1}" for i in range(len(split_docs))]
# print(ids_list)

try:
    # 添加文档到集合
    # collection.add(documents=split_docs)

    from itertools import batched
    docs_batches = batched(content_list, 3)  # 分3个批次
    ids_batches = batched(ids_list, 3)

    for docs, ids in zip(docs_batches, ids_batches):
        collection.add(
            documents=content_list,  # 传入纯文本列表
            metadatas=metadatas, 
            ids=ids_list
        )
    
    # 持久化向量数据库
    vectordb.persist()
    print("向量数据库已成功持久化到磁盘。")
except Exception as e:
    print(f"持久化过程中发生错误: {e}")

## 连接chroma数据库

In [ ]:
# 数据库路径 ./chroma2

In [ ]:
import chromadb
chroma_client  = chromadb.PersistentClient(path="../chroma-vmax")

In [ ]:
chroma_client.list_collections()

In [ ]:
# 获取一个存在的Collection对象
collection = chroma_client.get_collection("vmax-s")

In [ ]:
collection.count()  #  returns the number of items in the collection.

In [ ]:
# collection.peek()  # returns a list of the first 10 items in the collection.

In [ ]:
# 获取所有数据（默认不返回嵌入向量）
all_data = collection.get()
print(all_data)


## 向量检索
### 3.1 相似度检索
Chroma的相似度搜索使用的是余弦距离，即：
$$
similarity = cos(A, B) = \frac{A \cdot B}{\parallel A \parallel \parallel B \parallel} = \frac{\sum_1^n a_i b_i}{\sqrt{\sum_1^n a_i^2}\sqrt{\sum_1^n b_i^2}}
$$
其中$a_i$、$b_i$分别是向量$A$、$B$的分量。

当你需要数据库返回严谨的按余弦相似度排序的结果时可以使用`similarity_search`函数。

使用Chroma.from_documents时，用这个方法： 

In [ ]:
question="vmax"

In [ ]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

### 3.2 MMR检索
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。

核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [ ]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

In [ ]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")